In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
path = "/Users/trinakarmakar/anaconda2/data/fastai/rnn/nietzsche.txt"
text = open(path).read()
print('corpus length:', len(text))

corpus length: 529746


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1
print('total chars:', vocab_size)

total chars: 109


In [5]:
chars.insert(0, "\0")

In [6]:
''.join(chars[1:-6])

'\n !"$%&\'()*,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_abcdefghijklmnopqrstuvwxyz{|~\x80\x84\x94\x96\x99\x9e\xa0\xa2\xa3\xa5\xa6'

In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [8]:
idx = [char_indices[c] for c in text]

In [9]:
idx[:10]

[50, 73, 71, 78, 2, 82, 77, 2, 75, 63]

In [10]:
''.join(indices_char[i] for i in idx[:70])

'Skip to main content\nweb\ntexts\nmovies\naudio\nsoftware\nimage\nlogosearch\n'

In [11]:
#Create input:

cs=3 # we are going to predict every 4th character
c1_dat = [idx[i] for i in xrange(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in xrange(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in xrange(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in xrange(0, len(idx)-1-cs, cs)]

In [12]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

In [13]:
y = np.stack(c4_dat[:-2])

In [14]:
x1[:4], x2[:4], x3[:4]

(array([50, 78, 77, 63]), array([73,  2,  2, 71]), array([71, 82, 75, 76]))

In [15]:
y[:3]

array([78, 77, 63])

In [16]:
x1.shape, y.shape

((176579,), (176579,))

In [17]:
n_fac = 42

In [18]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [19]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

In [20]:
n_hidden=256

In [21]:
dense_in = Dense(n_hidden, activation='relu') # just creating a layer with a random weight matrix with 256 activations.

In [ ]:
#First hidden layer
c1_hidden_1 = dense_in(c1) # Activation is applied to first input c1 

In [ ]:
dense_hidden = Dense(n_hidden, activation='tanh') #New dense layer with different nonlinearity

In [ ]:
#This would be second hidden layer
c1_hidden_2 = dense_hidden(c1_hidden_1) # Apply new dense layer with previous output

In [ ]:
c2_hidden_1 = dense_in(c2) #Apply first activation to c2
c2_hidden_2 = merge([c2_hidden_1, c1_hidden_2]) # summing up two activation output

In [ ]:
c3_hidden_1 = dense_in(c3)
c2_hidden_3 = dense_hidden(c2_hidden_2)
c3_hidden_2 = merge([c3_hidden_1, c2_hidden_3])

In [ ]:
#output layer
dense_out = Dense(vocab_size, activation='softmax')

In [ ]:
c4_out = dense_out(c3_hidden_2)

In [ ]:
model = Model([c1_in, c2_in, c3_in], c4_out)

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
#model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=1)

In [ ]:
#Test Model:

def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i - np.argmax(p)
    return chars[i]

In [ ]:
get_next('phi')

In [ ]:
get_next(' th')

In [ ]:
get_next(' an')

Our First RNN: Yahoooooo

For each 0 through 7 create a list of 8th character with the that starting point. These will be 8 inputs our model.

In [26]:
cs = 8
c_in_data = [[idx[i+n] for i in xrange(0, len(idx) -1 -cs, cs)] for n in range(cs)]

Then creata a list of the next character in each of these series, these will be lavels of our model.

In [27]:
c_out_data = [idx[i+cs] for i in xrange(0, len(idx)-1-cs, cs)]

In [28]:
xs = [np.stack(c[:-2]) for c in c_in_data]

In [29]:
len(xs), xs[0].shape

(8, (66216,))

In [30]:
y = np.stack(c_out_data[:-2])

In [31]:
#so each column in this list is 8 characters from the text
[xs[n][:cs] for n in range(cs)]

[array([50, 75, 82,  1, 77, 66, 85, 69]),
 array([73, 63, 67, 82, 84, 71, 63, 67]),
 array([71, 71, 76, 67, 71, 77, 80,  1]),
 array([78, 76, 82, 86, 67,  1, 67, 74]),
 array([ 2,  2,  1, 82, 81, 81,  1, 77]),
 array([82, 65, 85, 81,  1, 77, 71, 69]),
 array([77, 77, 67,  1, 63, 68, 75, 77]),
 array([ 2, 76, 64, 75, 83, 82, 63, 81])]

In [32]:
#And this is the 8th character for each seqeunce
y[:cs]

array([75, 82,  1, 77, 66, 85, 69, 67])

In [33]:
n_fac = 42

In [34]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [ ]:
n_hidden= 256

In [ ]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

The first character of each sequence goes through dense_in(), to create out first hidden activations

In [ ]:
hidden = dense_in(c_ins[0][1])

Then for the each successive layer we combine the output of dense_in() on the next character with the output
dense_hidden() on the current hidden state, to create the new hidden state. 

In [ ]:
for i in range(1, cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

In [ ]:
c_out = dense_out(hidden)

So now we can create our model

In [ ]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
#model.fit(xs, y, batch_size=64, nb_epoch=12)
model.fit(xs, y, batch_size=64, nb_epoch=1)

In [ ]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [ ]:
get_next('for thos')

In [ ]:
get_next('part of ')

In [ ]:
get_next('queens a')

Our first RNN with Keras:

In [ ]:
#n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 110)

In [ ]:
vocab_size

In [ ]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
        Dense(vocab_size, activation='softmax')
    ])

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=Adam())

In [35]:
xs = np.stack(xs)

In [36]:
xs = xs.reshape(xs.shape[0], xs.shape[1])

In [37]:
np.stack(xs,1).shape

(66216, 8)

In [38]:
y.shape

(66216,)

In [ ]:
model.fit(np.stack(xs,1), y, batch_size=64, nb_epoch=12)

In [ ]:
def get_next_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = np.array(idxs)[np.newaxis,:]
    p = model.predict(arrs)[0]
    return chars[np.argmax(p)]

In [ ]:
get_next_keras('this is ')

In [ ]:
get_next_keras('part of ')

In [ ]:
get_next_keras('queens a')

Returning to Seqeunces:

To use seqence model we can leave input unchanged but need to change output to a sequence
c_out_dat will be like c_in_dat just moved one character 

In [39]:
#c_in_data = [[idx[i+n] for i in xrange(0, len(idx) -1 -cs, cs)] for n in range(cs)]
c_out_data = [[idx[i+n] for i in xrange(0, len(idx) - cs, cs)] for n in range(cs)]

In [40]:
ys = [np.stack(c[:-2]) for c in c_out_data]

In [41]:
#Reading down each column as one set of input and output
[xs[n][:cs] for n in range(cs)]

[array([50, 75, 82,  1, 77, 66, 85, 69]),
 array([73, 63, 67, 82, 84, 71, 63, 67]),
 array([71, 71, 76, 67, 71, 77, 80,  1]),
 array([78, 76, 82, 86, 67,  1, 67, 74]),
 array([ 2,  2,  1, 82, 81, 81,  1, 77]),
 array([82, 65, 85, 81,  1, 77, 71, 69]),
 array([77, 77, 67,  1, 63, 68, 75, 77]),
 array([ 2, 76, 64, 75, 83, 82, 63, 81])]

In [42]:
[ys[n][:cs] for n in range(cs)]

[array([50, 75, 82,  1, 77, 66, 85, 69]),
 array([73, 63, 67, 82, 84, 71, 63, 67]),
 array([71, 71, 76, 67, 71, 77, 80,  1]),
 array([78, 76, 82, 86, 67,  1, 67, 74]),
 array([ 2,  2,  1, 82, 81, 81,  1, 77]),
 array([82, 65, 85, 81,  1, 77, 71, 69]),
 array([77, 77, 67,  1, 63, 68, 75, 77]),
 array([ 2, 76, 64, 75, 83, 82, 63, 81])]

In [ ]:
#Create and train model:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax', name='output')   

In [ ]:
#as a starting point we pass all zeros as input
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

In [ ]:
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

In [ ]:
outs = []
for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden], mode='sum')
    # every layer now has an output
    outs.append(dense_out(hidden))

In [ ]:
model = Model([inp1] + [c[0] for c in c_ins], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]),1))
zeros.shape

In [ ]:
model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=12)

In [ ]:
#test model
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [ ]:
get_nexts(' this is')
get_nexts(' part of')
get_next('queens a')

Sequence model with keras

In [1]:
n_hidden, n_fac, cs, vocab_size

NameError: name 'n_hidden' is not defined

To convert our previous keras model into a sequence model, simply add the 'return_sequences=True' parameter, and add TimeDistributed() around our dense layer.

In [50]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, return_sequences=True, activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [51]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_13 (Embedding)         (None, 8, 42)         4578        embedding_input_2[0][0]          
____________________________________________________________________________________________________
simplernn_1 (SimpleRNN)          (None, 8, 256)        76544       embedding_13[0][0]               
____________________________________________________________________________________________________
timedistributed_2 (TimeDistribut (None, 8, 109)        28013       simplernn_1[0][0]                
Total params: 109135
____________________________________________________________________________________________________


In [52]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=Adam())

In [53]:
xs[0].shape

(66216,)

In [54]:
x_rnn = np.stack(xs, axis=1)
y_rnn = np.expand_dims(np.stack(ys, axis=1), -1)

In [55]:
x_rnn.shape, y_rnn.shape

((66216, 8), (66216, 8, 1))

In [56]:
model.fit(x_rnn, y_rnn, batch_size = 64, nb_epoch=2)

Epoch 1/2
66216/66216 [==============================] - 21s - loss: 0.4261       128/66216 [..............................] - ETA: 21s - loss: 4.6636

In [57]:
def get_next_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [59]:
get_next_keras(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


[' ', 't', 'h', 'i', 's', ' ', 'i', 's']

One hot seqeunce model with keras

A stateful model is easy to create (just add "stateful=True") but harder to train. We had to add batchnorm and use LSTM to get reasonable results.
When using stateful in keras, you have to also add 'batch_input_shape' to the first layer, and fix the batch size there.

In [60]:
bs  = 64

In [63]:
model=Sequential([
        SimpleRNN(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [64]:
oh_ys = [to_categorical(o, vocab_size) for o in ys]
oh_y_rnn=np.stack(oh_ys, axis=1)

oh_xs = [to_categorical(o, vocab_size) for o in xs]
oh_x_rnn=np.stack(oh_xs, axis=1)

oh_x_rnn.shape, oh_y_rnn.shape

((66216, 8, 109), (66216, 8, 109))

In [ ]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, nb_epoch=2)

Epoch 1/2
66216/66216 [==============================] - 21s - loss: 0.3914       128/66216 [..............................] - ETA: 19s - loss: 4.6496  640/66216 [..............................] - ETA: 24s - loss: 4.1470

In [ ]:
def get_nexts_oh(inp):
    idxs  np.array([char_indices[c] for c in inp])
    arr = to_categorical(idxs, vocab_size)
    p = model.predict(arr[np.newaxis,:])[0]
    print(list(inp))
    return [chars[np.argmax(0)] for in p]

In [ ]:
get_nexts_oh(' this is')

Stateful model with keras

In [ ]:
bs = 64

A stateful model is easy to create but harder to train, we have to add bacthnorm and use MSTM to get reasonable result

In [ ]:
model = Sequenctial([
    Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs,8)),
    BatchNormalization(),
    LSTM(n_hidden, return_sequences=True, stateful=True),
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])

In [ ]:
model.compile(loss='categorical_crossentriopy', optimizer=False)

In [62]:
mx = len(x_rnn)//bs*bs

In [ ]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

In [ ]:
model.optimizer.lr=1e-4

In [ ]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Theano RNN

In [ ]:
n_input = vocab_size
n_output = vocab_size

Using raw theano we need to create our own weight and bias metrices - we will use glorot initialization

In [ ]:
def init_wgts(rows, cols):
    scale = match.sqrt(2/rows)
    return shared(normal(scale=scale, size=(rows, cols)).astype(np.float32))
def init_bias(rows):
    return shared(np.zeros(rows, np.float32))